In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore


from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import os
import re

from dotenv import load_dotenv
load_dotenv()

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

In [2]:
def get_pdf_file_names(folder_path):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.

    Args:
        folder_path (str): PDF 파일이 있는 폴더의 경로.

    Returns:
        list: PDF 파일 이름들의 리스트.
    """
    try:
        # 폴더 내 파일 리스트 가져오기
        all_files = os.listdir(folder_path)
        
        # 확장자가 '.pdf'인 파일만 필터링
        pdf_files = [file.replace(".pdf","") for file in all_files if file.lower().endswith('.pdf')]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [3]:
def load_and_split_tax_law(file_path,repeat_pattern):
    """
    PDF 파일을 로드하고, 반복 텍스트 제거, 조항 기준으로 분할 및 연결 후 임베딩 준비.
    Args:
        file_path (str): PDF 파일 경로.
    Returns:
        list: 연결된 텍스트 조각들의 리스트.
    """
    import re
    from langchain.document_loaders import PyMuPDFLoader
    from langchain_openai import OpenAIEmbeddings

    # PDF 로드
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    
    # 전체 텍스트 가져오기
    full_text = " ".join([doc.page_content for doc in documents])

    # 1. 반복 텍스트 제거
    full_text = re.sub(repeat_pattern, "", full_text)

    # 2. '제n조'를 기준으로 분할
    split_pattern = r"\n(제\d+조[^\s]*)"
    chunks = re.split(split_pattern, full_text)

    # 3. '제n조'와 연결하여 완전한 문단 구성 + 부칙 처리
    connected_chunks = []
    current_chunk = ""
    is_buchik_section = False  # 부칙 여부 플래그

    for chunk in chunks:
        if re.search(r"\n\s*<부칙>", chunk):  # 부칙 시작 여부 확인
            is_buchik_section = True

        if chunk.startswith("제") and "조" in chunk:  # 새로운 조항 시작
            if is_buchik_section:  # 부칙 섹션이라면 접두사 추가
                chunk = "부칙-" + chunk

            if current_chunk:  # 이전 조항 저장
                connected_chunks.append(current_chunk.strip())
            current_chunk = chunk  # 새 조항으로 시작
        else:
            current_chunk += f" {chunk}"  # 기존 조항에 내용 추가

    if current_chunk:
        connected_chunks.append(current_chunk.strip())


    return connected_chunks, full_text



In [ ]:
def get_vector_store(pdf_files):
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.

    Args:
        pdf_files (list): PDF 파일 이름들의 리스트.
        vector_store_path (str): Chroma Vector Store를 저장할 경로.
    """
    from langchain_chroma import Chroma
    from langchain_core.documents import Document
    
    # 각 PDF 파일에 대해 임베딩 처리
    all_docs = []
    
    for file in pdf_files:
        path = f"data/tax_law/{file}.pdf"  # PDF 파일 경로
        repeat_pattern = rf"법제처\s*\d+\s*국가법령정보센터\n{file.replace('_', ' ')}\n" 
        chunks, full_text = load_and_split_tax_law(path, repeat_pattern)
        

        for idx, chunk in enumerate(chunks):
            _doc = Document(metadata={"title": file, "full_text": full_text[idx], }, page_content=chunk),
            all_docs.extend(_doc)

    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    vector_store = Chroma.from_documents(
        documents=all_docs,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
        
    return vector_store
        
folder_path = "data/tax_law" 
pdf_files = get_pdf_file_names(folder_path)

In [ ]:
# 📌 중복 저장 될 수 있어서 주석 처리함!
# vector_store = get_vector_store(pdf_files)

In [12]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# vector_store =  Chroma(
#     collection_name=COLLECTION_NAME,
#     persist_directory=PERSIST_DIRECTORY,
#     embedding_function=embedding_model,
# )

In [7]:
vector_store._collection.count()

5733

In [8]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":10}
)


In [9]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser



In [10]:
# chain.invoke("개별소비세법이 뭐야?")
# chain.invoke("개별소비세법이 제1조가 뭐야")
# chain.invoke("교통_에너지_환경세법 뭐야?")
chain.invoke("조세범 처벌절차법의 정의해줘")


'조세범 처벌절차법에서 사용하는 용어의 정의는 다음과 같습니다:\n\n1. **조세범칙행위**: 이는 「조세범 처벌법」 제3조부터 제16조까지의 죄에 해당하는 위반행위를 말합니다.\n   \n2. **조세범칙사건**: 조세범칙행위의 혐의가 있는 사건을 의미합니다.\n\n3. **조세범칙조사**: 세무공무원이 조세범칙행위 등을 확정하기 위하여 조세범칙사건에 대하여 행하는 조사활동을 말합니다.\n\n4. **세무공무원**: 세무에 종사하는 공무원으로서, 지방국세청 소속 공무원이나 세무서 소속 공무원의 경우 각각 소속 지방국세청장의 제청으로 해당 지방국세청이나 세무서의 소재지를 관할하는 지방검찰청의 검사장이 지명하는 공무원을 의미합니다.\n\n이 정의들은 조세범 처벌절차법 제2조에 명시되어 있습니다. 추가적으로 궁금한 점이 있으면 말씀해 주세요!'